### Artificial Neural Networks (Multi-Layer Perceptron)
#### Amaç, en küçük hata ile tahmin yapabilecek katsayılara erişmektir.
##### İnsan beyninin bilgi işleme şeklini referans alan sınıflandırma ve regresyon problemlerinde kullanılabilen kuvvetli makine öğrenmesi algoritmalarından birisidir.

In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("diabetes.csv")

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#### Model & Tuning

In [6]:
mlpc_model = MLPClassifier().fit(X_train, y_train)

In [7]:
mlpc_model.coefs_

[array([[-5.31824940e-02,  3.41825133e-02, -1.08463283e-01,
         -1.39716622e-01, -1.11729066e-01,  9.75151847e-02,
         -5.83731789e-02, -1.42027820e-01,  6.51387343e-02,
          9.01492717e-03, -1.37965217e-01, -2.38950092e-01,
         -1.75374631e-01, -9.73527430e-02, -1.52300783e-02,
         -8.08534045e-02,  2.31503984e-02,  4.43561232e-02,
         -1.47420971e-02,  7.66183018e-02,  2.29646264e-02,
          4.90777594e-02,  5.65280494e-02,  1.28457866e-01,
          1.14559208e-01,  1.74415775e-01,  2.15564432e-01,
          7.63932008e-02,  1.08572730e-01, -2.22739512e-03,
         -6.27869321e-02,  2.47228536e-01,  1.54162660e-01,
          8.67656248e-03, -1.84913035e-01, -2.45644440e-02,
          7.34458957e-02, -2.85758035e-01,  9.84206481e-04,
          7.29373915e-02,  2.01771177e-01, -9.25719969e-03,
          2.42464095e-02, -1.51667425e-01, -2.32117383e-02,
          1.67417954e-01, -7.03409732e-02, -2.50181062e-01,
          1.18518919e-01, -7.42325159e-0

###### mlpc_model: Bu model log-loss fonksiyonunu optimize eder. 
###### log-loss: Gerçek değerler ile tahmin edilen değerler arasındaki farklara ilişkin optimizasyon yapmak.
###### solver: Modelde kullanılan ağırlıkları optimize etmeye yarar. Varsayılan:adam, veri seti daha küçükse lbfgs kullanmak daha mantıklıdır.
###### activation: varsayılan değer: relu'dur. Doğrusalproblemlerde relu, sınıflandırmada logistic kullanılır.
###### alpha: ceza terimi(lambda).
###### 

In [10]:
y_pred = mlpc_model.predict(X_test)

In [11]:
accuracy_score(y_test, y_pred)

0.7012987012987013

#### Model Tuning

In [13]:
mlpc_params = {"alpha": [0.1, 0.01, 0.03, 0.005, 0.0001, 1, 5],
               "hidden_layer_sizes": [(10,10), (100,100,100), (100,100), (3,5)]}

In [21]:
mlpc = MLPClassifier(solver="lbfgs", activation="logistic")

In [22]:
mlpc_cv_model = GridSearchCV(mlpc, mlpc_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 28 candidates, totalling 280 fits


In [23]:
mlpc_cv_model.best_params_

{'alpha': 5, 'hidden_layer_sizes': (100, 100)}

In [27]:
# Final Model
mlpc_tuned = MLPClassifier(solver="lbfgs", activation="logistic", alpha=5, hidden_layer_sizes=(100,100)).fit(X_train, y_train)

In [28]:
y_pred = mlpc_tuned.predict(X_test)

In [29]:
accuracy_score(y_test, y_pred)

0.7402597402597403

#### Scaling

In [30]:
scaler = StandardScaler()

In [31]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [33]:
scaler.fit(X_test)
X_test = scaler.transform(X_test)

In [34]:
mlpc_params = {"alpha": [0.1, 0.01, 0.03, 0.005, 0.0001, 1, 5],
               "hidden_layer_sizes": [(10,10), (100,100,100), (100,100), (3,5)]}

In [35]:
mlpc = MLPClassifier(solver="lbfgs", activation="logistic")

In [36]:
mlpc_cv_model = GridSearchCV(mlpc, mlpc_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 28 candidates, totalling 280 fits


In [37]:
mlpc_cv_model.best_params_

{'alpha': 1, 'hidden_layer_sizes': (3, 5)}

In [38]:
# Final Model
mlpc_tuned = MLPClassifier(solver="lbfgs", activation="logistic", alpha=1, hidden_layer_sizes=(3,5)).fit(X_train, y_train)

In [39]:
y_pred = mlpc_tuned.predict(X_test)

In [40]:
accuracy_score(y_test, y_pred)

0.7575757575757576